In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

os.environ['SNORKELDB'] = 'sqlite:///Users/bach/snorkel/tutorials/snark/snorkel'

In [3]:
from snorkel import SnorkelSession
snorkel = SnorkelSession()

OperationalError: (sqlite3.OperationalError) unable to open database file

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Snark Tutorial").master("spark://bachmacbook:7077").getOrCreate()

In [4]:
from snorkel.models import candidate_subclass
Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

OperationalError: (sqlite3.OperationalError) unable to open database file

In [5]:
from snorkel.contrib.snark import SparkLabelAnnotator

annotator = SparkLabelAnnotator(snorkel, spark, Spouse)

In [6]:
def LF_1(c):
    return 1

In [12]:
annotator.apply(LFs=[LF_1], split=0)

Py4JJavaError: An error occurred while calling o191.load.
: java.sql.SQLException: opening db: '//./snorkel.db': Permission denied
	at org.sqlite.core.CoreConnection.open(CoreConnection.java:203)
	at org.sqlite.core.CoreConnection.<init>(CoreConnection.java:76)
	at org.sqlite.jdbc3.JDBC3Connection.<init>(JDBC3Connection.java:26)
	at org.sqlite.jdbc4.JDBC4Connection.<init>(JDBC4Connection.java:24)
	at org.sqlite.SQLiteConnection.<init>(SQLiteConnection.java:45)
	at org.sqlite.JDBC.createConnection(JDBC.java:114)
	at org.sqlite.JDBC.connect(JDBC.java:88)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$$anonfun$createConnectionFactory$1.apply(JdbcUtils.scala:59)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$$anonfun$createConnectionFactory$1.apply(JdbcUtils.scala:50)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.resolveTable(JDBCRDD.scala:58)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRelation.<init>(JDBCRelation.scala:113)
	at org

In [ ]:
import os